In [1]:
import numpy as np
import pandas as pd
import os

In [17]:
data_path = "/Users/sergmiller/Documents/my/IDAO-2021/data/idao_dataset/"

In [18]:
train_path = os.path.join(data_path, "train")

In [19]:
train_NR = os.path.join(train_path, "NR")
train_ER = os.path.join(train_path, "ER")

In [28]:
def read_all_png_in_dir(path : str, limit : int = None) -> dict:
    from os import listdir
    from os.path import isfile, join
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import tqdm

    imgs = listdir(path)
    imgs = map(lambda f: join(path, f), imgs)
    imgs = filter(lambda f: isfile(f) and '.png' == f[-4:], imgs)
    imgs = list(imgs)
    if limit is not None:
        imgs = imgs[:limit]
    data = {img : mpimg.imread(img) for img in tqdm.tqdm(imgs, position=0)}
    return data

In [29]:
sample_train_NR_imgs = read_all_png_in_dir(train_NR, 10)

100%|██████████| 10/10 [00:00<00:00, 130.60it/s]


In [30]:
sample_train_ER_imgs = read_all_png_in_dir(train_ER, 10)

100%|██████████| 10/10 [00:00<00:00, 130.19it/s]


In [31]:
sample_train_NR_imgs

{'/Users/sergmiller/Documents/my/IDAO-2021/data/idao_dataset/train/NR/-0.301800012588501__CYGNO_60_40_He_NR_1_keV_930V_30cm_SRIM_IDAO_iso_crop_hist_pic_run3_ev372;1.png': array([[0.4       , 0.40784314, 0.40392157, ..., 0.42352942, 0.39215687,
         0.39215687],
        [0.41960785, 0.4       , 0.39607844, ..., 0.4       , 0.38431373,
         0.3882353 ],
        [0.39215687, 0.38431373, 0.4       , ..., 0.4117647 , 0.39607844,
         0.38431373],
        ...,
        [0.4       , 0.42745098, 0.39607844, ..., 0.3882353 , 0.39215687,
         0.38039216],
        [0.4       , 0.3764706 , 0.39215687, ..., 0.41568628, 0.39607844,
         0.3882353 ],
        [0.4       , 0.38039216, 0.39607844, ..., 0.4117647 , 0.3882353 ,
         0.38431373]], dtype=float32),
 '/Users/sergmiller/Documents/my/IDAO-2021/data/idao_dataset/train/NR/0.6492999792098999__CYGNO_60_40_He_NR_6_keV_930V_30cm_SRIM_IDAO_iso_crop_hist_pic_run2_ev479;1.png': array([[0.38431373, 0.3882353 , 0.39215687, ..., 0.36

In [32]:
class LabeledDataset:
    def __init__(self):
        self.samples = []
        self.labels = []
        self.tags = []

    def add_sample(self, tag : str, sample : np.array, label : np.array):
        self.samples.append(img)
        self.labels.append(label)
        self.tags.append(tag)
        
    def finalize(self):
        self.samples = np.array(self.samples)
        self.labels = np.array(self.labels)
        self.tags = np.array(self.tags)

In [33]:
from itertools import chain

In [34]:
for x in chain([1,2,3],[4,5,6]):
    print(x)

1
2
3
4
5
6


In [ ]:
def build_particles_dataset(path : dir, limit : int = None) -> LabeledDataset:
    ER_path = os.path.join(path, "ER")
    NR_path = os.path.join(path, "NR")
    ER_samples = read_all_png_in_dir(ER_path, limit)
    NR_samples = read_all_png_in_dir(NR_path, limit)
    